In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread

In [2]:
import rankingmethod
import rankingmeasure
import imp
imp.reload(rankingmethod)
imp.reload(rankingmeasure)

<module 'rankingmeasure' from '/home/george/Documents/diploma/code/rankingmeasure.py'>

In [3]:
tmp = mmread("../datasets/epinions_original/EP25_UPL5_test.mtx")
test = np.array(list(zip(tmp.row, tmp.col)))
tmp = mmread("../datasets/epinions_original/EP25_UPL5_train.mtx")
train = np.array(list(zip(tmp.row, tmp.col)))
user_item = [np.max()]

In [4]:

print(np.unique(train[:,1]).shape[0])
print(np.unique(train[:,0]).shape[0])
print(np.unique(test[:, 1]).shape[0])
print(np.unique(test[:, 0]).shape[0])

8121
4718
34928
4718


In [208]:
# b = loadmat("../datasets/epinions/trustnetwork.mat")
# data = np.array(b["trustnetwork"])

In [196]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [303]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25
#Также проводится переиндексанция пользователей и предметов

In [8]:
def create_listarray(data):
    list_good_user = list(np.where(np.bincount(data[:, 0]) > 25)[0])
    set_good_user = set(list_good_user)
    epinion = []
    for i,j in data:
        if i in set_good_user:
            epinion.append([i, j])
    epinion = np.array(epinion)
    for new_u, u in enumerate(np.sort(list_good_user)):
        epinion[epinion[:, 0] == u, 0] = new_u
    print(epinion.shape)
    list_good_item = np.unique(epinion[:, 1])
    for new_i, i in enumerate(np.sort(list_good_item)):
        epinion[epinion[:, 1] == i, 1] = new_i
    user_item = np.max(epinion, axis=0) + 1
    return epinion, user_item

In [149]:
def givenK_train_test(data, K):
    xlist = []
    for u in np.sort(np.unique(data[:, 0])):
        xlist.append(data[data[:, 0] == u,  1])
    
    
    train = []
    test = []
    for x in xlist:
        train_ind = np.random.choice(x.shape[0], K, replace=False)
        tmp = np.ones(x.shape[0])
        tmp[train_ind] = 0
        test_ind = np.arange(x.shape[0])[tmp == 1]
        
        train.append(x[train_ind])
        test.append(x[test_ind])
        
    return (train, test, xlist)

In [293]:
train, test, traintest = givenK_train_test(epinion, 5)

In [15]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = rankmethod.get_list(0)[:3]

NameError: name 'user_item' is not defined

In [324]:
poprec.fit(train)

In [363]:
randomrec = rankingmethod.RandomRec(user_item)
randomrec.fit(train)
all_measures(train, test, randomrec, True, top)

precision5 0.00614107883817
1-call5 0.00459197786999
MRR 0.027050101239855347


In [327]:
all_measures(train, test, poprec, True, top)

precision5 0.0665006915629
1-call5 0.0665006915629
MRR 0.13060863758010338


In [320]:
def all_measures(train, test, method, skip_train=False, top=None):
    print("precision5", rankingmeasure.get_prec_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("1-call5", rankingmeasure.get_prec_K(5, train, test, 
            method, skip_train=skip_train, top=top))
    print("MRR", rankingmeasure.get_MRR(train, test, 
            method, skip_train=skip_train, top=top))

In [345]:
list_reg = [0.01, 0.1 ,0.5]
list_lrate = [0.001, 0.005, 0.01, 0.05, 0.1 ]
list_K = [10, 5, 7, 12, 15]

for K in list_K:
    for lrate in list_lrate:
        for reg in list_reg:
            climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=100)
            climf.fit(train)
            print(K, lrate, reg)
            print(rankingmeasure.get_MRR(train, test, 
            climf, skip_train=False, top=top))

10 0.001 0.01
0.0068061377633100405
10 0.001 0.1
0.008031961359121426
10 0.001 0.5
0.005912269091311202
10 0.005 0.01
0.007815140434096543
10 0.005 0.1
0.00652022706541012
10 0.005 0.5
0.006532552664074765
10 0.01 0.01
0.007028481810001253
10 0.01 0.1
0.005199687139427464
10 0.01 0.5
0.00642130218487009
10 0.05 0.01
0.007314500601830544
10 0.05 0.1
0.00563555140338767
10 0.05 0.5
0.005847528094546522
10 0.1 0.01
0.008851060117673922
10 0.1 0.1
0.008597647640724496
10 0.1 0.5
0.007773293622440457
5 0.001 0.01
0.008755247464899283
5 0.001 0.1
0.010599960371667312
5 0.001 0.5
0.006476902717603026
5 0.005 0.01
0.010910411353978392
5 0.005 0.1
0.005833571238977291
5 0.005 0.5
0.008890794054437025
5 0.01 0.01
0.015242684443990519
5 0.01 0.1
0.006276795759708992
5 0.01 0.5
0.005705023945597651
5 0.05 0.01
0.022483016113061575
5 0.05 0.1
0.005442549602997411
5 0.05 0.5
0.008115720609231233
5 0.1 0.01
0.019875807579590635
5 0.1 0.1
0.009115952585898184
5 0.1 0.5
0.01036981495442007
7 0.001 0.01

KeyboardInterrupt: 

In [359]:
a = np.arange(5)
print(a)
print(np.random.shuffle(a))
print(a)

[0 1 2 3 4]
None
[3 2 4 0 1]


In [340]:
all_measures(train, test, climf, True, top)

precision5 0.00110650069156
1-call5 0.00110650069156
MRR 0.007748599151933581
